## Using generateDS on REG AB II
2020-09-07

The script generateDS.py generates Python code from an xsd. This code can be a good starter to use instead of the xml library for parsing xml, to take advantage of all the info in the xsd. We'll use the [generateDS](https://www.davekuhlman.org/generateDS.html) to read an XML Auto Loan file [Version 1,8](https://www.sec.gov/info/edgar/specifications/absxml.htm), since that's the xml I am wrestling with. 

After installing (see instructions at link), the CL used is below. 
- generateDS -o "autoLoan.py" -s "autoSub.py" eis_ABS_AutoLoanAssetData.xsd
Conveniently, the script stores the CL in the generated code, so you can always check to see what you ran. 

In [1]:
import pandas as pd
from datetime import datetime as dt
import sys
import os
import importlib
from collections import OrderedDict
import urllib.request
import requests


import autoSub

### Constants
All the files but the xml used here are in the repo. Names below. The xml file is too big to store on gh and we'll have to pull a copy over from Edgar. There is a copy as well on S3 in case Edgar reorgs this someplace else. 

In [2]:
XSDN = "eis_ABS_AutoLoanAssetData.xsd" # autoLoan schema (from Edgar)
LWBN = "dtypes.xls" # xls layout (retyped)

GC1N = "genAutoLoan.py" # geneted code file
GC2N = "genAutoSub.py" # additional generated code file
INFN = "20200706.AfsSensub.xml" # one submission for a month for one SPE

# S3 copy of this file in case Edgar reorrgs
# RABD = "https://datadeloro0tutorials.s3-us-west-2.amazonaws.com/regAB" # directory
# INFP = "/".join((RABD, "xml", INFN))

EDGR = "https://www.sec.gov/Archives/edgar/data"
INFP = "/".join((EDGR, "1347185/000134718520000016/exh1024080052020.xml"))

#### Tabled version of layout
I hand copied the PDF layout from [Edgar](https://www.sec.gov/info/edgar/specifications/absxml.htm) and made some notes that will help with using the data later. Browse if interested, but we can read directly into Pandas! Here, we are just using the list of fieldnames to keel things in order. 

In [3]:
fmtR = pd.read_excel(LWBN).set_index("index")
fmtR.shape

(72, 4)

In [4]:
fmtR.head()

,fieldName,dType,maxLength,encoding
index,,,,
1,assetTypeNumber,str,100,NaN
2,assetNumber,str,25,NaN
3,reportingPeriodBeginningDate,Mm-dd-yyyy,NaN,NaN
4,reportingPeriodEndingDate,Mm-dd-yyyy,NaN,NaN
5,originatorName,str,50,NaN


### Generate code with generateDS
Assumed that that has been installed somewhere on path.

In [5]:
os.system('generateDS -o "%s" -s "%s" eis_ABS_AutoLoanAssetData.xsd' %(GC1N, GC2N))

0

#### Minor edit to one of the generated files
The "???" in GC2N needs to be replaced with GC1N (without the .py)

In [6]:
os.system("cp %s deleteMe.py" %(GC2N))
newF = open(GC2N, "w")
for line in open("deleteMe.py"):
    newF.write(line.replace("???", GC1N.replace(".py", "")))
newF.close()

#### Use importlib to import generate code using full path

In [7]:
spec = importlib.util.spec_from_file_location(GC2N.replace(".py", "") \
                                              , os.path.join(os.getcwd(), GC2N))
genClass = importlib.util.module_from_spec(spec) 
spec.loader.exec_module(genClass)

### Read the data file
Now we have a genClass that was made from the code generated from the xsd. Below we use that class to load the data into a structure. We'll watch the time also. 

In [8]:
# grab data file
# urllib.request.urlretrieve(INFP, INFN)
req = requests.get(INFP)
inF = open(INFN, "wb")
inF.write(req.content)
inF.close()

In [9]:

print (dt.now().strftime("%H:%M:%S"))
root = autoSub.parse(INFN, silence=True)
print (dt.now().strftime("%H:%M:%S"))
len(root.assets)

17:54:57
17:55:13


55887

#### Comparing to xml lib
Lust looking at one obs to get a feel for what using the xsd to generate code bought us. 

In [10]:
root.assets[0].__dict__["assetNumber"] 

'0001814917 - 000001'

#### Repeating with xml lib to compare
Parsing with xml.etree.ElementTree is about 2x faster (because Python modules are almost always faster than Python), but the names in the structure have garbage in them I just don't want to deal with. 

In [11]:
import xml.etree.ElementTree as ET
print (dt.now().strftime("%H:%M:%S"))
tree = ET.parse(INFN)
fromXmlLib = tree.getroot()
print (dt.now().strftime("%H:%M:%S"))
len(fromXmlLib)

17:55:30
17:55:38


55887

In [12]:
print (fromXmlLib[0][1])
print (fromXmlLib[0][1].text)

<Element '{http://www.sec.gov/edgar/document/absee/autoloan/assetdata}assetNumber' at 0x7ff918646220>
0001814917 - 000001


### Converting to DataFrame
Because the parser in the generated code gives the same names as the documentation, we can just use those names to build a dictionary and cast it as a DF. Using an OrderedDict here to give the DF the same order as the docs. 

In [13]:
# init dict
tD = OrderedDict() 
for field in fmtR.fieldName:
    tD[field] = [] # set up list for an element
# populate
for entry in root.assets:
    for field in fmtR.fieldName:
        tD[field].append(entry.__dict__[field])
# cast as DF
tR = pd.DataFrame(tD)
tR.shape

(55887, 72)

#### Lists
There are a few fields in the Version 1.8 spec that can occur an "unlimited" number of times, which was probably a bad idea but the generated parser handles well.

In [14]:
mnL = """subvented zeroBalanceCode repurchaseReplacementReasonCode 
modificationTypeCode""".split()
mL = [tR.iloc[0][mn] for mn in mnL]
print (mL)
print (list(map(type, mL)))

[['0'], [], [], []]
[<class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>]


### Checking and Postprocessing
The parser has validators, but, since we have the data now loaded into a DF, I prefer to validate there. Because this file has all valid values for the reportingPeriodBeginningDate, we can cast it to a date with a Pandas to_datetime. Validting and casting all 72 fields for all SPE is a bigger exercise. 

In [15]:
print (tR.reportingPeriodBeginningDate.value_counts().sort_index())  # before 
tR.reportingPeriodBeginningDate = pd.to_datetime(tR.reportingPeriodBeginningDate)
print (tR.reportingPeriodBeginningDate.value_counts().sort_index()) # after

05-01-2020    55887
Name: reportingPeriodBeginningDate, dtype: int64
2020-05-01    55887
Name: reportingPeriodBeginningDate, dtype: int64
